#TRABAJO PRÁCTICO 1 - SPARK
 - Alumno: Daniela Ojeda.
 - Padrón: 107690.
 - Cuatrimestre: 1C2023.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
#!apt install default-jre
#!apt install default-jdk+
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=a6a059b41d64ebf00d07a088cd0927a061399fd3ce2e2bbf9d424bc11833cf30
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark
Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Get:4 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/c

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd
import math

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

##EJERCICIO 9

In [ ]:
'''Mostrar de forma eficiente el tercer trigrama que tiene mayor frecuencia en los títulos de los contenidos de la wikipedia.'''
downloaded = drive.CreateFile({'id':"1h9Lrqmc4uAkXeC0DJAg3f4VlJLut5C7S"})
downloaded.GetContentFile('contents.csv')
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)
df_contents = sqlContext.read.csv('/content/drive/MyDrive/orga/wikipedia/contents.csv', header=True, inferSchema=True)
rdd_contents = df_contents.rdd

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [ ]:
rdd_filtro = rdd_contents.filter(lambda x: x.title is not None and len(x.title) > 2)
rdd_separo_palabras = rdd_filtro.flatMap(lambda x: x[0].lower().split()).filter(lambda x: len(x) > 2)
rdd_separo_trigramas = rdd_separo_palabras.flatMap(lambda x: [x[i:i+3] for i in range(len(x) -2)]).filter(lambda x: x.isalpha() and len(x) == 3)
rdd_obtengo_frecuencias = rdd_separo_trigramas.map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b)
rdd_top_3_trigramas = rdd_obtengo_frecuencias.takeOrdered(3,key=lambda x: -x[1])
tercer_trigrama = rdd_top_3_trigramas[2]
print(f'El tercer trigrama que tiene mayor frecuencia en los títulos de los contenidos de la wikipedia es: {tercer_trigrama[0]} con {tercer_trigrama[1]} apariciones')

El tercer trigrama que tiene mayor frecuencia en los títulos de los contenidos de la wikipedia es: ría con 483092 apariciones


##EJERCICIO 19

In [ ]:
'''Cantidad de contenido por planeta fuera de la tierra en la Wikipedia.'''
downloaded = drive.CreateFile({'id':"1h9Lrqmc4uAkXeC0DJAg3f4VlJLut5C7S"})
downloaded.GetContentFile('geo_tags.csv')
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)
df_geo_tags = sqlContext.read.csv('/content/drive/MyDrive/orga/wikipedia/geo_tags.csv', header=True, inferSchema=True)
rdd_geo_tags = df_geo_tags.rdd

In [ ]:
contenido_fuera_tierra = rdd_geo_tags.filter(lambda x: x.gt_globe != 'earth' and x.gt_globe is not None).map(lambda x: (x.gt_globe, 1)).reduceByKey(lambda a,b: a+b).collect()
print(f'La cantidad de contenido por planeta fuera de la Tierra en la Wikipedia es: ')
for i in contenido_fuera_tierra:
  print(f'• {i[0]} - {i[1]} contenidos')

La cantidad de contenido por planeta fuera de la Tierra en la Wikipedia es: 
• mars - 1438 contenidos
• titan - 4 contenidos
• callisto - 3 contenidos
• titania - 2 contenidos
• hyperion - 1 contenidos
• tethys - 1 contenidos
• mimas - 1 contenidos
• mercury - 561 contenidos
• moon - 9061 contenidos
• ganymede - 116 contenidos
• venus - 24 contenidos
• umbriel - 1 contenidos
• phobos - 20 contenidos
• oberon - 1 contenidos
• io - 2 contenidos
• deimos - 2 contenidos
• enceladus - 1 contenidos


##EJERCICIO 22

In [ ]:
'''Top 5 de lenguajes que son usados por usuarios bilingües'''
downloaded = drive.CreateFile({'id':"1h9Lrqmc4uAkXeC0DJAg3f4VlJLut5C7S"})
downloaded.GetContentFile('languages.csv')
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)
df_languages = sqlContext.read.csv('/content/drive/MyDrive/orga/wikipedia/languages.csv', header=True, inferSchema=True)
rdd_languages = df_languages.rdd

In [ ]:
rdd_languages_new = rdd_languages.map(lambda x: (x.babel_user, x.babel_lang, '4.5') if x.babel_level == 'N' else x)
rdd_languages_pasaje_int = rdd_languages_new.map(lambda x: (x[0], x[1], float(x[2])))

rdd_languages_bilingues_idiomas = rdd_languages_pasaje_int.filter(lambda x: x[2] >= 1).map(lambda x: (x[0], [x[1].lower()])).reduceByKey(lambda a,b: a + b).filter(lambda x: len(x[1]) > 1)
rdd_languages_contador_idiomas = rdd_languages_bilingues_idiomas.flatMap(lambda x: [(idioma, 1) for idioma in set(x[1])]).reduceByKey(lambda a,b: a + b)
top_5_usados = rdd_languages_contador_idiomas.takeOrdered(5, key=lambda x: -x[1])
print('Los 5 idiomas más usados por usuarios bilingües son:')
for i in top_5_usados:
  print(f'• {i[0]} ({i[1]} usuarios)')


Los 5 idiomas más usados por usuarios bilingües son:
• en (9318 usuarios)
• es (9200 usuarios)
• fr (3476 usuarios)
• it (1738 usuarios)
• de (1642 usuarios)


##EJERCICIO 24

In [ ]:
'''Dado un tamaño de vocabulario parametrizable y una lista de stopwords también parametrizable implemente tf-IDF para los textos de los contenidos de
forma distribuida. Debe obtener un vector por cada texto'''
downloaded = drive.CreateFile({'id':"1h9Lrqmc4uAkXeC0DJAg3f4VlJLut5C7S"})
downloaded.GetContentFile('contents_text_sample.csv')
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)
df_contents_text_sample = sqlContext.read.csv('/content/drive/MyDrive/orga/wikipedia/contents_text_sample.csv', header=True, inferSchema=True)
rdd_contents_text_sample = df_contents_text_sample.rdd

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [ ]:
!pip install nltk
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

#Ya se que este ejercicio no esta bien hecho pero me costo mucho plantearlo y preferi entregar algo aunque no funcione, antes que no entregar nada
#Uso esto de ejemplo a ver si funciona
#stopwordsEsp = stopwords.words('spanish')
#vocabulario_size = 10

def tokenizar(texto, stopwords):
  tokens = word_tokenize(texto.lower())
  tokens = [token for token in tokens if token.isalnum() and token not in stopwords]
  return tokens

rdd_contents_text_sample = rdd_contents_text_sample.filter(lambda x: x[2] is not None).map(lambda x: (x[1],tokenizar(x[2], stopwordsEsp)))
rdd_textos = rdd_contents_text_sample.flatMap(lambda x: x[1]).distinct().take(vocabulario_size)

def calculo_tf(termino, tokens):
  return len(tokens.filter(termino))/len(tokens)

def contienen_termino(termino, rdd):
  return len(rdd.map(lambda x: tokenizar(x[2], stopwordsEsp)).filter(lambda x: termino in x))

def calculo_idf(termino, rdd):
  return math.log(len(rdd.text)/(1+contienen_termino(termino,rdd)))

def formar_vector(tokens, rdd, rdd2):
  vector = []
  for termino in rdd2:
    tf = calculo_tf(termino, tokens)
    idf = calculo_idf(termino,rdd)
    vector.append(tf*idf)
  return vector

rdd_vectores = rdd_contents_text_sample.map(lambda x: (x[0], formar_vector(x[1], rdd_contents_text_sample, rdd_textos)))
rdd_vectores.take(5)